In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

void reset_gpio() {
    write_gpio(1,0);
    write_gpio(2,0);
    write_gpio(3,0);
    write_gpio(0,0);
}


In [3]:

reset_gpio()
import asyncio
stop = False
begin = True
#frequency must be nonzero
def pwm(frequency, duty_cycle, pin):
    global stop
    if (frequency == 0):
        print("frequency cannot be 0")
        return
    
    if duty_cycle == 0: 
        write_gpio(pin,0)
    elif duty_cycle == 100:
        write_gpio(pin,1)
    else:
        period = 1/frequency
        time_on = period*duty_cycle*.01
        time_off = period - time_on
        while(True):
            write_gpio(pin,1)
            time.sleep(time_on)
            write_gpio(pin,0)
            time.sleep(time_off)
            if btns.read() : 
                reset_gpio()
                break

#gpio mappings:
#1 = blue
#2 = red
#3 = green
async def get_btns(_loop):
    global stop, begin
    while True:
        await asyncio.sleep(0.01)
        if begin: 
            pwm(.5, 25, 2)
            begin = False
        
        elif btns.read() == 4:
        #blue
            pwm(1,25,1)
        elif btns.read() == 2:
        #green
    
            pwm(1,25,3)
            
        elif btns.read() == 1:
        #red 
           
            pwm(1,25,2)

In [ ]:
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.run_forever()

In [4]:
reset_gpio()

In [9]:
btns.read()

8